In [11]:
from pathlib import Path
import polars as pl

In [12]:
ROOT_DIR = Path("/home/bobby/repos/latent-neural-dynamics-modeling")
saved_dir = ROOT_DIR / "data"

In [13]:
participants = pl.read_parquet(
    saved_dir / "resampled" / "participants_intermediate.parquet"
)

In [14]:
participants = participants.with_columns(pl.col("run").alias("block"))

In [15]:
behav_features = pl.read_parquet(saved_dir / "PDI_behav_features.parquet")

In [16]:
behav_features = behav_features.with_columns(
    pl.concat_str(
        pl.lit("sub-"),
        pl.col("subject"),
    ).alias("participant_id"),
    pl.col("session").cast(pl.UInt64),
    pl.col("ix_block").cast(pl.UInt64).alias("block"),
    pl.col("ix_trial").cast(pl.UInt64).alias("trial"),
).drop("subject", "ix_block", "ix_trial")

In [17]:
behav_features

trial_time,pre_trial_time,is_fragmented,delta_t,speed,acceleration,velocity_x,velocity_y,acceleration_x,acceleration_y,isj_x,isj_y,isj,speed_sub,acceleration_sub,velocity_x_sub,velocity_y_sub,acceleration_x_sub,acceleration_y_sub,isj_x_sub,isj_y_sub,isj_sub,pathlen,dt,dt_l,dist_sub,dt_norm,len,speed_sub_bin_1,accel_sub_bin_1,jerk_sub_bin_1,dist_sub_bin_1,speed_sub_bin_2,accel_sub_bin_2,jerk_sub_bin_2,dist_sub_bin_2,speed_sub_bin_3,accel_sub_bin_3,jerk_sub_bin_3,dist_sub_bin_3,speed_sub_bin_4,accel_sub_bin_4,jerk_sub_bin_4,dist_sub_bin_4,speed_sub_bin_5,accel_sub_bin_5,jerk_sub_bin_5,dist_sub_bin_5,speed_sub_bin_6,accel_sub_bin_6,jerk_sub_bin_6,dist_sub_bin_6,speed_sub_bin_7,accel_sub_bin_7,jerk_sub_bin_7,dist_sub_bin_7,speed_sub_bin_8,accel_sub_bin_8,jerk_sub_bin_8,dist_sub_bin_8,file,session,stim,stim_ampl,stim_unit,exclude_due_to_protocol,participant_id,block,trial
f64,f64,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,u64,str,str,str,bool,str,u64,u64
9.0,2.06486,false,0.008419,410.837504,91821.078717,142.444444,352.666667,31924.82716,78630.888889,36831.0,196740.0,116785.5,407.175488,4112.749537,141.623924,350.899056,1809.076291,3253.702747,1990.0,5234.0,3612.0,1068,15767.369865,15767.369865,9.536891,14.749644,2.043977,5.12,0.02,-0.06,10.783,7.795455,-0.155172,-0.267442,9.024766,4.241379,0.086207,0.137931,11.228896,-1.0,0.625,0.375,8.870217,-4.153846,0.269231,-0.076923,9.115025,-8.5,-0.032258,-0.032258,8.081716,-3.089744,0.102564,0.00641,10.345023,1.151163,-0.232558,0.27907,9.565822,"""/Volumes/MDExternM2/data/copyd…",2,"""off""","""6""","""mA""",false,"""sub-PDI1""",11,2
9.0,1.222764,false,0.008515,377.505752,81456.580148,138.333333,317.333333,29830.888889,68391.814815,30075.0,158082.0,94078.5,371.865975,3601.791331,136.963068,313.961494,1528.235924,2852.41762,1514.0,4849.0,3181.5,1056,13694.030133,13694.030133,8.861952,12.955563,2.021033,4.733333,-0.533333,-0.25,7.556671,7.109756,0.036585,-0.140244,9.118215,3.758621,0.293103,-0.103448,10.053308,-0.5,0.5,0.5,9.659114,-3.388889,0.194444,-0.027778,9.49299,-7.5,0.139535,-0.023256,9.556393,-3.142857,-0.035714,0.228571,8.629078,1.25,0.15,-0.133333,7.962857,"""/Volumes/MDExternM2/data/copyd…",2,"""off""","""6""","""mA""",false,"""sub-PDI1""",11,3
9.0,1.333596,false,0.008563,353.964917,74118.243025,108.666667,308.555556,23277.703704,64383.481481,25128.0,151753.0,88440.5,349.95284,3688.394655,106.464071,307.304105,1433.206836,2991.603426,1384.0,4875.0,3129.5,1050,12261.300701,12261.300701,8.443573,11.666318,2.00956,5.636364,-0.204545,0.204545,9.521074,6.590361,-0.096386,-0.168675,8.426759,3.958333,-0.145833,0.166667,9.257507,-2.5,2.0,-1.0,8.946067,null,null,null,null,-7.423077,0.118421,0.026316,7.619868,-3.343284,-0.089552,0.089552,8.219744,1.425,-0.2875,0.05,8.7232,"""/Volumes/MDExternM2/data/copyd…",2,"""off""","""6""","""mA""",false,"""sub-PDI1""",11,8
9.0,1.231359,false,0.008419,374.100145,81990.612253,126.666667,317.555556,27582.839506,69880.925926,28498.0,162935.0,95716.5,369.365973,3879.993452,125.542899,314.411765,1470.972227,3135.146776,1300.0,5928.0,3614.0,1068,13276.868268,13276.868268,8.62665,12.419895,2.043977,3.903226,-0.133333,-0.034483,8.51291,7.277778,-0.145833,-0.180556,8.669787,3.560606,0.090909,0.045455,8.787711,-0.75,0.25,0.125,7.941394,-5.5625,0.3125,0.1875,8.492045,-8.257143,-0.042857,0.085714,9.004286,-3.32716,-0.049383,0.111111,8.794755,1.448276,-0.155172,0.034483,8.669316,"""/Volumes/MDExternM2/data/copyd…",2,"""off""","""6""","""mA""",false,"""sub-PDI1""",11,4
9.0,1.887891,false,0.008483,400.808285,87381.884257,141.888889,339.444444,31188.160494,73798.444444,36587.0,194610.0,115598.5,395.9668,3909.790511,139.757167,336.571668,1683.787504,3084.035661,1665.0,5593.0,3629.0,1060,12007.372551,12007.372551,7.755852,11.317034,2.028681,4.808824,-0.264706,0.088235,7.124108,8.5,

In [18]:
participants = participants.join(
    behav_features, ["participant_id", "session", "block"], "left"
)

In [19]:
participants = participants.select(
    pl.col("participant_id"),
    pl.col("session"),
    pl.col("block"),
    pl.col("trial"),
    pl.col("onsets"),
    pl.col("trial_time"),
    pl.col("is_fragmented"),
    pl.col("stim"),
    pl.col("exclude_due_to_protocol"),
    pl.col("ieeg_headers_file"),
    pl.col("session_path"),
).unique()

In [20]:
participants

participant_id,session,block,trial,onsets,trial_time,is_fragmented,stim,exclude_due_to_protocol,ieeg_headers_file,session_path
str,u64,u64,u64,list[f64],f64,bool,str,bool,str,str
"""sub-PDI4""",2,13,12,"[60.065909, 93.506727, … 360.924]",9.0,false,"""on""",false,"""/Users/matthias.dold/workspace…","""/Users/matthias.dold/workspace…"
"""sub-PDI2""",2,6,3,"[9.472, 23.947682, … 181.275273]",9.0,false,"""on""",false,"""/Users/matthias.dold/workspace…","""/Users/matthias.dold/workspace…"
"""sub-PDI2""",2,1,9,"[11.898273, 28.406273, … 187.821045]",9.0,false,"""off""",false,"""/Users/matthias.dold/workspace…","""/Users/matthias.dold/workspace…"
"""sub-PDI1""",4,9,10,"[11.117182, 28.178045, … 201.150182]",9.0,false,"""off""",false,"""/Users/matthias.dold/workspace…","""/Users/matthias.dold/workspace…"
"""sub-PDI1""",4,9,4,"[11.117182, 28.178045, … 201.150182]",9.0,false,"""off""",false,"""/Users/matthias.dold/workspace…","""/Users/matthias.dold/workspace…"
…,…,…,…,…,…,…,…,…,…,…
"""sub-PDI3""",2,3,7,"[9.739545, 25.624409, … 184.712]",9.0,true,"""off""",false,"""/Users/matthias.dold/workspace…","""/Users/matthias.dold/workspace…"
"""sub-PDI1""",4,2,12,"[47.559091, 66.656591, … 264.633364]",9.0,false,"""on""",false,"""/Users/matthias.dold/workspace…","""/Users/matthias.dold/workspace…"
"""sub-PDI3""",2,5,3,null,9.0,false,"""off""",false,"""/Users/matthias.dold/workspace…","""/Users/matthias.dold/workspace…"


In [21]:
participants = (
    participants.with_columns(pl.col("participant_id").str.split(by="-").list.get(-1))
    .with_columns(
        pl.col("ieeg_headers_file")
        .str.split("/")
        .list.get(-1)
        .str.split(".")
        .list.get(0)
        .alias("base_ieeg_file")
    )
    .with_columns(
        pl.concat_str(
            pl.lit(str(saved_dir / "resampled")),
            pl.concat_str(pl.col("base_ieeg_file"), pl.lit("parquet"), separator="."),
            separator="/",
        ).alias("ieeg_parquet"),
        pl.col("session").cast(pl.UInt32),
        pl.col("session_path")
        .str.split(by="/")
        .list.tail(2)
        .list.join(separator="/")
        .map_elements(lambda s: str(saved_dir.joinpath(s)), return_dtype=pl.String)
        .alias("session_path"),
    )
    .drop("base_ieeg_file", "ieeg_headers_file", "durations", "run", strict=False)
)

participants = participants.drop(
    "participant_path", "ieeg_path", "ieeg_file", strict=False
).unique()

In [22]:
participants = participants.filter(exclude_due_to_protocol=False).drop(
    "exclude_due_to_protocol"
)

In [23]:
participants

participant_id,session,block,trial,onsets,trial_time,is_fragmented,stim,session_path,ieeg_parquet
str,u32,u64,u64,list[f64],f64,bool,str,str,str
"""PDI1""",2,1,2,"[18.102864, 36.896455, … 213.614364]",9.0,false,"""off""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI4""",3,3,12,"[12.149727, 26.195182, … 184.120727]",9.0,false,"""off""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI1""",2,6,7,"[15.075818, 33.513409, … 208.159227]",9.0,false,"""on""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI2""",2,2,4,"[11.209, 26.374045, … 189.906545]",9.0,false,"""on""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI3""",2,6,6,"[16.681591, 32.142182, … 197.628318]",9.0,false,"""on""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
…,…,…,…,…,…,…,…,…,…
"""PDI2""",2,4,8,"[12.711045, 28.169273, … 176.29]",9.0,false,"""on""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI1""",4,4,11,"[17.497545, 34.855318, … 204.154136]",9.0,false,"""on""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI4""",3,3,1,"[12.149727, 26.195182, … 184.120727]",9.0,false,"""off""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"


In [24]:
# group by participant_id, session, block
# put trials into a list, the rest is overalpping, meaning that the columns the same for all trials in a block so maybe into a set and then explode

participants = (
    participants.group_by(["participant_id", "session", "block"])
    .agg(
        pl.col("trial").alias("trials"),
        pl.col("onsets").first().alias("onsets"),
        pl.col("trial_time").first().alias("trial_time"),
        pl.col("is_fragmented").first().alias("is_fragmented"),
        pl.col("stim").first().alias("stim"),
        pl.col("ieeg_parquet").first().alias("ieeg_parquet"),
        pl.col("session_path").first().alias("session_path"),
    )
    .with_columns(pl.col("trials").list.sort())
)

In [25]:
participants

participant_id,session,block,trials,onsets,trial_time,is_fragmented,stim,ieeg_parquet,session_path
str,u32,u64,list[u64],list[f64],f64,bool,str,str,str
"""PDI1""",4,8,"[1, 2, … 12]","[22.761773, 40.423955, … 213.960182]",9.0,false,"""on""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI2""",2,1,"[1, 2, … 12]","[11.898273, 28.406273, … 187.821045]",9.0,false,"""off""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI3""",4,3,"[1, 2, … 12]","[12.227955, 29.381682, … 199.263364]",9.0,true,"""off""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI2""",2,2,"[1, 2, … 12]","[11.209, 26.374045, … 189.906545]",9.0,false,"""on""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI3""",4,5,"[1, 2, … 12]","[10.946318, 30.516909, … 206.079455]",9.0,false,"""off""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
…,…,…,…,…,…,…,…,…,…
"""PDI1""",2,9,"[1, 2, … 12]","[24.195182, 39.663773, … 202.610773]",9.0,false,"""off""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI2""",2,5,"[1, 2, … 12]","[9.993182, 25.765182, … 181.805409]",9.0,false,"""off""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"
"""PDI2""",2,11,"[1, 2, … 12]","[9.037318, 24.680182, … 173.733409]",9.0,false,"""off""","""/home/bobby/repos/latent-neura…","""/home/bobby/repos/latent-neura…"


In [27]:
participants.group_by(["participant_id", "session", "block", "trials"]).len()

participant_id,session,block,trials,len
str,u32,u64,list[u64],u32
"""PDI3""",4,1,"[1, 2, … 12]",1
"""PDI2""",2,12,"[1, 2, … 12]",1
"""PDI2""",4,7,"[1, 2, … 12]",1
"""PDI3""",4,5,"[1, 2, … 12]",1
"""PDI4""",2,8,"[1, 2, … 12]",1
…,…,…,…,…
"""PDI2""",4,6,"[1, 2, … 12]",1
"""PDI1""",2,12,"[1, 2, … 12]",1
"""PDI4""",2,12,"[1, 2, … 12]",1


In [81]:
participants.write_parquet(
    saved_dir / "participants_2",
    partition_by=["participant_id", "session", "block"],
)